<a href="https://colab.research.google.com/github/JADukeAIPI/AIPIProject/blob/main/WebScrapingAtlantaEvents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import json


In [2]:
today_date = datetime.today().strftime('%Y-%m-%d')

def get_end_date(number_days):

  end_date = datetime.today() + timedelta(days=number_days)
  end_date=end_date.strftime('%Y-%m-%d')
  return end_date

end_date = get_end_date(90)


In [24]:
def get_events(start_date = today_date, end_date = get_end_date(90), query_string = None):
    '''
    Scrapes data from Creative Loafing, Google events, and Discover Atlanta for the next time period and organizes the data in a DataFrame

    Inputs:
        Date range in terms of strings of YYYY-MM-DD for today and future date; default is today to today + 90 days
        Query string (default None) which will perform .query(query_string) on the dataframe

    Returns:
        events_df(DataFrame): DataFrame containing the scraped data for all upcoming events in the specified date range
    '''
    
    # URLS to search  
    #url = 'https://creativeloafing.com/atlanta-events/this-month' 
    #url2 ='https://www.google.com/search?q=atlanta+convention+center+events&oq=atlanta+convention+center+events&aqs=chrome..69i57j0i512j0i22i30l6.4881j0j7&sourceid=chrome&ie=UTF-8&ibp=htl;events&rciv=evn&sa=X&ved=2ahUKEwjIsY_c4J_7AhX-lWoFHWBFBBcQ66QDKAZ6BAgIEAw#htichips=date:month&htischips=date;month&htidocid=L2F1dGhvcml0eS9ob3Jpem9uL2NsdXN0ZXJlZF9ldmVudC8yMDIyLTExLTExfDI1MTcwNDI2NDQ0NjM3MjA4ODg%3D&htivrt=events&fpstate=tldetail'
    url3 = 'https://discoveratlanta.com/events/all/?startdate='+today_date+'&enddate='+(end_date)+'&date_sort=this-month'
    #url4 = 'https://www.google.com/search?lei=3P9qY9XjLZGvqtsPpuOckAE&q=atlanta+convention+calendar+2022&biw=1440&bih=764&dpr=2&ibp=htl;events&rciv=evn&sa=X&ved=2ahUKEwil94el95_7AhUZibAFHSDpDQkQ8eoFKAJ6BAgJEA8#htivrt=events&htidocid=L2F1dGhvcml0eS9ob3Jpem9uL2NsdXN0ZXJlZF9ldmVudC8yMDIyLTExLTExfDI1MTcwNDI2NDQ0NjM3MjA4ODg%3D&fpstate=tldetail'

    # Request the page and use BeautifulSoup to extract the contents
    #page = requests.get(url)
    #page2 = requests.get(url2)
    page3 = requests.get(url3)
    #page4 = requests.get(url4)
    
    #soup = BeautifulSoup(page.content, 'html.parser')
    #soup2 = BeautifulSoup(page2.content, 'html.parser')
    soup3 = BeautifulSoup(page3.content, 'html.parser')
    #soup4 = BeautifulSoup(page4.content, 'html.parser')

    #for now, just using url3
    events = soup3.find_all('article', class_='listing')
    
    event_dict={} #to store event info
    for event in events:
      title = event.find(attrs={'class':'listing-title'}).text
      dates = json.loads(event['data-eventdates'])
      location = event.find(attrs={'class':'subtitle location'}).text
      description = event.find(attrs={'class':"listing-copy detail"}).text     
      event_dict[title]={'Dates': dates, 'Location': location, 'Description': description}
 
    #create dataframe
    event_df = pd.DataFrame.from_dict(event_dict).T
    event_df.reset_index(inplace=True)
    event_df = event_df.rename(columns = {'index':'Event'})
    event_df = event_df.explode('Dates', ignore_index=True)
    event_df['Dates'] = pd.to_datetime(event_df['Dates'])
    event_df = event_df.sort_values(by='Dates', ascending=True).reset_index(drop=True)
    event_df = event_df[(event_df['Dates'] > start_date) & (event_df['Dates'] <= end_date)]
    
    if query_string is not None:
      event_df = event_df.query(query_string, engine='python')

    return event_df

    raise NotImplementedError()

In [25]:
events_data_df = get_events()

In [26]:
events_data_df

,Event,Dates,Location,Description
1532,Delicious Thanksgiving Offerings from Fox Bros...,2022-11-14,Fox Bros. Bar-B-Q,Fox Bros. Bar-B-Q welcomes another year of hol...
1533,The Colors of Peace Exhibit,2022-11-14,Jimmy Carter Presidential Library & Museum,Bringing together 270+ artworks created by chi...
1534,Yu-Gi-Oh! Mondays,2022-11-14,Caffeinated Gamers,Official (but friendly) Event! We have Yu-Gi-O...
1535,"Garden Lights, Holiday Nights",2022-11-14,"Atlanta Botanical Garden, Inc.","The 12th annual Garden Lights, Holiday Nights ..."
1536,CFHOF Honors 2022 Inductees with New Exhibit,2022-11-14,Chick-fil-A College Football Hall of Fame,
...,...,...,...,...
3774,Duke Ellington’s Cat,2023-02-10,Center For Puppetry Arts,A Musical Giant & his Cool Cat Written and dir...
3775,Stephen Burks Shelter in Place,2023-02-11,High Museum of Art,"In the wake of several global crises, designer..."
3776,Duke Ellington’s Cat,2023-02-11,Center For Puppetry Arts,A Musical Giant & his Cool Cat Written and dir...
3777,Midtown Atlanta Food and Cocktail Tour,2023-02-11,Saint Mark United Methodist Church,Indulge in an eclectic array of cuisine & cock...


In [27]:
large_venue_query = 'Location.str.contains("State Farm Arena|Mercedes-Benz Stadium")'
large_venue_events = get_events(query_string=large_venue_query)
large_venue_events


,Event,Dates,Location,Description
1588,Kingdom Tour: Maverick City Music & Kirk Franklin,2022-11-15,State Farm Arena,Influential gospel acts Kirk Franklin and Mave...
1626,Atlanta Hawks vs. Boston Celtics,2022-11-16,State Farm Arena,Come cheer on the Hawks as they take on the Bo...
1729,DAVIDO: ARE WE AFRICAN YET? Music Festival,2022-11-18,State Farm Arena,"Afrobeats superstar, recording artist, produce..."
1758,Atlanta Hawks vs. Toronto Raptors,2022-11-19,State Farm Arena,Come cheer on the Hawks as they take on the To...
1813,Atlanta Falcons vs. Chicago Bears – Nov. 20,2022-11-20,Mercedes-Benz Stadium,Come watch the Atlanta Falcons take on the Chi...
1909,ATEEZ – The Fellowship: Break The Wall Tour,2022-11-22,State Farm Arena,K-pop giants ATEEZ have announced a monumental...
1941,Atlanta Hawks vs. Sacramento Kings,2022-11-23,State Farm Arena,Come cheer on the Hawks as they take on the Sa...
2168,Atlanta Hawks vs. Miami Heat,2022-11-27,State Farm Arena,Come cheer on the Hawks as they take on the Mi...
2304,Rod Wave – Beautiful Mind Tour,2022-12-01,State Farm Arena,Platinum-selling rapper Rod Wave has announced...
2332,Disney On Ice: Frozen & Encanto,2022-12-02,State Farm Arena,"For the first time, Disney On Ice invites fami..."
